In [14]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import talib
import matplotlib
from secrets import IEX_CLOUD_API_TOKEN
import csv
from pandas_datareader import data
from datascience import *
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
%matplotlib inline

In [15]:
ticker = "TSLA"
url = f"https://sandbox.iexapis.com/stable/stock/{ticker}/chart/5y/?token={IEX_CLOUD_API_TOKEN}"
stock_data = requests.get(url).json()
print(stock_data)

[{'close': 50.713, 'high': 52.101, 'low': 48.619, 'open': 52.101, 'symbol': 'TSLA', 'volume': 58341521, 'id': 'C_LTIRIAEHIOPSCRS', 'key': 'ALST', 'subkey': '', 'date': '2017-02-27', 'updated': 1657763597079, 'changeOverTime': 0, 'marketChangeOverTime': 0, 'uOpen': 253.9, 'uClose': 246.65, 'uHigh': 255.16, 'uLow': 248.54, 'uVolume': 11647656, 'fOpen': 49.93, 'fClose': 49.658, 'fHigh': 50.003, 'fLow': 49.504, 'fVolume': 59655072, 'label': 'Feb 27, 17', 'change': 0, 'changePercent': 0}, {'close': 50.29, 'high': 51.9, 'low': 50.29, 'open': 51.198, 'symbol': 'TSLA', 'volume': 30545541, 'id': '_OPHSACRLCTRIEIIS', 'key': 'LTSA', 'subkey': '', 'date': '2017-02-28', 'updated': 1653534454428, 'changeOverTime': 0.01594809663474038, 'marketChangeOverTime': 0.015312263012748076, 'uOpen': 252.63, 'uClose': 256.6, 'uHigh': 257, 'uLow': 252.9, 'uVolume': 6297047, 'fOpen': 50.81, 'fClose': 51.659, 'fHigh': 50.4, 'fLow': 50.11, 'fVolume': 31014887, 'label': 'Feb 28, 17', 'change': 0.7780426269792091, 'c

In [16]:
full_dataframe = pd.DataFrame(stock_data)
full_dataframe

,close,high,low,open,symbol,volume,id,key,subkey,date,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
0,50.713,52.1010,48.6190,52.101,TSLA,58341521,C_LTIRIAEHIOPSCRS,ALST,,2017-02-27,...,248.5400,11647656,49.930,49.658,50.0030,49.5040,59655072,"Feb 27, 17",0.000000,0.0000
1,50.290,51.9000,50.2900,51.198,TSLA,30545541,_OPHSACRLCTRIEIIS,LTSA,,2017-02-28,...,252.9000,6297047,50.810,51.659,50.4000,50.1100,31014887,"Feb 28, 17",0.778043,0.0159
2,52.195,52.6730,51.5300,52.673,TSLA,24212011,I_OIRICCRHASTLESP,ATSL,,2017-03-01,...,260.2400,4949908,50.892,50.263,51.0000,51.9910,24973878,"Mar 1, 17",0.006287,0.0001
3,50.972,51.7450,50.5200,51.714,TSLA,17487042,ICSREIIPTOACLHRS_,STAL,,2017-03-02,...,256.0800,3354203,50.866,50.991,52.1380,50.0660,17217633,"Mar 2, 17",0.093947,0.0019
4,52.825,52.8250,51.1260,51.126,TSLA,15090570,LOCIRIACSEISRP_TH,ASTL,,2017-03-03,...,250.0000,3019547,52.430,51.650,51.8000,51.0000,15117346,"Mar 3, 17",0.222903,0.0044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,863.540,921.0000,861.5200,921.000,TSLA,23508983,SISIO_ARRCPTILHCE,TASL,,2022-02-18,...,866.8700,22893520,901.000,886.860,891.8500,867.9800,23915272,"Feb 18, 22",-20.053516,-0.0226
1256,857.500,898.1104,817.2332,854.150,TSLA,27890579,OIIACSTH_RSECILRP,TASL,,2022-02-22,...,815.4419,27826230,836.930,855.620,886.0424,807.1748,28748971,"Feb 22, 22",-35.678971,-0.0418
1257,773.270,855.7400,773.2700,855.740,TSLA,33263773,TSIHRACPR_SCOIELI,ALST,,2022-02-23,...,796.9200,32370645,858.070,769.760,867.7621,788.0100,31890869,"Feb 23, 22",-59.368250,-0.0700
1258,820.060,838.8400,717.0500,717.050,TSLA,46027091,ISTIACPORRS_HLIEC,ALST,,2022-02-24,...,730.0000,46851191,728.400,803.470,842.0300,731.0000,46076178,"Feb 24, 22",37.234057,0.0493


In [17]:
full_dataframe['EMA30'] = talib.EMA(full_dataframe['close'].values, timeperiod=30)
full_dataframe['date'] = pd.to_datetime(full_dataframe['date'])
full_dataframe = full_dataframe.set_index("date")
print(full_dataframe.describe())

             close         high          low         open        volume  \
count  1260.000000  1260.000000  1260.000000  1260.000000  1.260000e+03   
mean    284.269316   292.330672   276.047210   284.114711  4.582649e+07   
std     327.304095   337.100377   317.723966   327.388331  3.084983e+07   
min      36.851000    38.310300    36.167000    37.710000  1.014599e+07   
25%      62.569500    64.099500    60.880000    62.496750  2.570168e+07   
50%      72.233500    73.948900    70.775500    72.750000  3.584695e+07   
75%     565.082500   582.852500   546.325000   552.497500  5.383126e+07   
max    1268.240000  1289.680000  1242.650000  1272.830000  3.100407e+08   

            updated  changeOverTime  marketChangeOverTime        uOpen  \
count  1.260000e+03     1260.000000           1260.000000  1260.000000   
mean   1.652519e+12        4.750738              4.751547   528.139294   
std    2.469722e+10        6.653335              6.661438   326.257737   
min    1.606847e+12       -0

In [18]:
monthly_avg = full_dataframe.resample('M').mean()
monthly_avg

,close,high,low,open,volume,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,change,changePercent,EMA30
date,,,,,,,,,,,,,,,,,,,,,
2017-02-28,50.501500,52.000500,49.454500,51.649500,4.444353e+07,1.655649e+12,0.007974,0.007656,253.265000,251.625000,...,250.720000,8.972352e+06,50.370000,50.658500,50.201500,49.807000,4.533498e+07,0.389021,0.007950,NaN
2017-03-31,52.917304,53.955626,51.935983,53.096870,2.385145e+07,1.645748e+12,0.049667,0.049560,264.696522,265.573043,...,262.375000,4.758325e+06,52.640870,52.749261,53.637735,52.209757,2.382552e+07,0.250334,0.004922,NaN
2017-04-30,62.341474,63.745737,61.188347,62.545632,3.157612e+07,1.648556e+12,0.243734,0.245386,312.955263,312.226316,...,308.824537,6.280959e+06,62.377158,62.256895,63.263858,61.377216,3.147587e+07,0.385926,0.006874,57.011399
2017-05-31,65.184955,66.174455,63.216636,64.481545,3.440819e+07,1.650210e+12,0.291789,0.293084,321.822955,324.542727,...,318.245105,6.906731e+06,64.524545,64.722818,65.661909,63.786273,3.455761e+07,0.245914,0.004314,62.204728
2017-06-30,74.921955,76.334486,73.296364,75.305273,4.331321e+07,1.645008e+12,0.497644,0.495540,376.435364,374.148182,...,366.941073,8.688055e+06,74.795318,75.065045,75.994273,73.817927,4.329993e+07,0.198309,0.002927,69.087391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-31,896.068095,913.215833,876.384171,893.990629,2.570049e+07,1.678247e+12,17.263621,17.349669,899.442224,898.780000,...,879.341329,2.583036e+07,891.224533,900.510952,914.139743,883.165533,2.571412e+07,16.394979,0.018086,802.844518
2021-11-30,1148.419524,1184.207271,1108.934286,1141.322810,3.164681e+07,1.678968e+12,22.327538,22.364913,1147.712524,1152.315714,...,1110.726667,3.167646e+07,1147.663190,1151.884286,1176.088648,1111.737619,3.159205e+07,1.380480,0.002314,1018.247969
2021-12-31,1042.978636,1075.580464,1009.320805,1049.445264,2.372699e+07,1.675949e+12,20.292877,20.285932,1050.302323,1044.554091,...,1019.766741,2.376378e+07,1047.917459,1047.100909,1073.229300,1017.357736,2.393652e+07,-4.050158,-0.002950,1058.859135


In [19]:
clean = monthly_avg.drop("changeOverTime", 1).drop("marketChangeOverTime", 1).drop("change", 1).drop("changePercent", 1).drop("updated", 1)
clean = clean.replace("-","", regex=True)
cleaner = clean.copy()
cleaner.dropna(inplace=True)
cleaner

,close,high,low,open,volume,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,EMA30
date,,,,,,,,,,,,,,,,
2017-04-30,62.341474,63.745737,61.188347,62.545632,3.157612e+07,312.955263,312.226316,314.941242,308.824537,6.280959e+06,62.377158,62.256895,63.263858,61.377216,3.147587e+07,57.011399
2017-05-31,65.184955,66.174455,63.216636,64.481545,3.440819e+07,321.822955,324.542727,328.632195,318.245105,6.906731e+06,64.524545,64.722818,65.661909,63.786273,3.455761e+07,62.204728
2017-06-30,74.921955,76.334486,73.296364,75.305273,4.331321e+07,376.435364,374.148182,381.189377,366.941073,8.688055e+06,74.795318,75.065045,75.994273,73.817927,4.329993e+07,69.087391
2017-07-31,67.361300,69.405335,65.960340,68.133450,4.661439e+07,340.300000,336.213500,343.704085,330.506605,9.384548e+06,68.028600,67.232500,68.873760,65.886860,4.633800e+07,70.141464
2017-08-31,72.011000,73.329270,70.497500,71.553826,3.025932e+07,359.120435,357.855652,364.337378,355.609278,6.086164e+06,71.909696,72.166783,72.528883,71.266574,3.059147e+07,70.571386
2017-09-30,73.212500,74.701475,71.937505,73.615800,2.854674e+07,368.413000,366.643000,371.778240,361.795945,5.703987e+06,73.573150,73.258850,74.197370,72.172370,2.865006e+07,72.451595
2017-10-31,70.638318,71.825236,69.006409,70.457318,2.868143e+07,353.144091,353.026818,355.750795,349.414045,5.716598e+06,70.785636,70.638864,71.500918,70.014473,2.856209e+07,71.948735
2017-11-30,63.642810,65.252157,62.140429,63.642857,3.633720e+07,317.869048,318.224286,323.364971,313.015714,7.279812e+06,63.669667,63.458381,64.439295,62.614762,3.646271e+07,67.065819
2017-12-31,66.103600,67.634295,64.636650,66.036400,2.823013e+07,331.197000,331.576000,335.286215,328.694500,5.612376e+06,66.024750,66.395200,66.880075,64.863750,2.805519e+07,65.752506


In [37]:
X = full_dataframe.drop("close", 1).replace("-","", regex=True).drop("EMA30", 1).drop("symbol", 1).drop("id", 1).drop("key", 1).drop("subkey", 1).drop("changeOverTime", 1).drop("marketChangeOverTime", 1).drop("change", 1).drop("changePercent", 1).drop("updated", 1).drop("label", 1)
y = full_dataframe["close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scale = StandardScaler()

In [38]:
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

In [39]:
mlp = MLPRegressor(activation = "tanh", hidden_layer_sizes=(15,15,15), max_iter=100000)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)

In [44]:
last_row = X[-1:]
print(last_row)
print(mlp.predict(last_row))

              high       low    open    volume   uOpen  uClose  uHigh  \
date                                                                    
2022-02-25  846.32  806.3862  846.32  25515126  843.74  842.95  851.1   

                uLow   uVolume   fOpen  fClose  fHigh      fLow   fVolume  
date                                                                       
2022-02-25  783.7036  25392071  820.32  845.64  857.4  797.2253  25547640  
[553.73948338]


In [50]:
X_clean = cleaner.drop("close", 1)
y_clean = cleaner["close"]
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size = 0.2, random_state = 42)
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)
avgmlp = MLPRegressor(activation = "tanh", hidden_layer_sizes=(15,15,15), max_iter=100000)
avgmlp.fit(X_train, y_train)
predictions = avgmlp.predict(X_test)

In [51]:
last_row = X_clean[-1:]
print(last_row)
print(avgmlp.predict(last_row))

                  high         low        open      volume       uOpen  \
date                                                                     
2022-02-28  934.280328  873.803856  902.270556  24488885.0  901.927778   

                uClose       uHigh        uLow       uVolume       fOpen  \
date                                                                       
2022-02-28  902.341667  924.437283  882.821711  2.446329e+07  901.596111   

            fClose       fHigh        fLow       fVolume       EMA30  
date                                                                  
2022-02-28  902.05  927.180144  876.125456  2.445948e+07  976.802235  
[933.0486652]
